In [ ]:
import MetaTrader5 as mt5
import pandas as pd
import plotly.graph_objects as go
from datetime import datetime
import os
from dotenv import load_dotenv

load_dotenv()

mt5_account = int(os.getenv('MT5_ACCOUNT'))
mt5_password = os.getenv('MT5_PASSWORD')
mt5_server = os.getenv('MT5_SERVER')
mt5_path = os.getenv('MT5_PATH')

if not mt5.initialize(path=mt5_path, login=mt5_account, password=mt5_password, server=mt5_server):
    print("initialize() failed, error code =", mt5.last_error())
account_info = mt5.account_info()
if account_info is None:
    print("Failed to connect to account, error code =", mt5.last_error())
else:
    print("Successfully connected to MetaTrader 5.")
    print("Account Info:", account_info.name)

symbol = "XAUUSD"
timeframe_h4 = mt5.TIMEFRAME_H4
timeframe_h1 = mt5.TIMEFRAME_H1
start_date = datetime(2023, 1, 1) 
end_date = datetime.now()         

h4_rates = mt5.copy_rates_range(symbol, timeframe_h4, start_date, end_date)
df_h4 = pd.DataFrame(h4_rates)
df_h4['time'] = pd.to_datetime(df_h4['time'], unit='s')
df_h4.set_index('time', inplace=True)
print(f"\nFetched {len(df_h4)} H4 bars for {symbol}.")
print("H4 Data Sample:")
print(df_h4.tail())

h1_rates = mt5.copy_rates_range(symbol, timeframe_h1, start_date, end_date)
df_h1 = pd.DataFrame(h1_rates)
df_h1['time'] = pd.to_datetime(df_h1['time'], unit='s')
df_h1.set_index('time', inplace=True)
print(f"\nFetched {len(df_h1)} H1 bars for {symbol}.")
print("H1 Data Sample:")
print(df_h1.tail())

fig = go.Figure(data=[go.Candlestick(x=df_h4.index,
                open=df_h4['open'],
                high=df_h4['high'],
                low=df_h4['low'],
                close=df_h4['close'])])

fig.update_layout(
    title=f'{symbol} H4 Candlestick Chart',
    xaxis_title='Date',
    yaxis_title='Price',
    xaxis_rangeslider_visible=False 
)
fig.show()

mt5.shutdown()
print("\nMetaTrader 5 connection shut down.")